# Project Time Series Notebook

In [1]:
import itertools
import reprlib
import numpy as np

class TimeSeries:
    '''
    List implementation of time series
    
    RepInv: Times and values must only include numbers, and must be same length.
    '''
    def __init__(self, times, values):
        self.repOK(times, values)
        
        # Sort times and values in ascending order of time - It's just neater that way
        times, values = (list(x) for x in zip(*sorted(zip(times, values), key=lambda pair: pair[0])))
        
        self.__timesseq = list(times)
        self.__valuesseq = list(values)
        self.__times_to_index = {t: i for i, t in enumerate(times)}
        
    def repOK(self, times, values):
        assert self._hasOnlyNumbers(times) and self._hasOnlyNumbers(values), "Both times and values should only include numbers"
        assert len(times) == len(values), "Length of times and values must be the same"
    
    def _hasOnlyNumbers(self, arr):
        '''
        Private function to test if the input array consists of only numbers
        '''
        for i in arr:
            try:
                int(i)
            except ValueError:
                return False
        return True
        
    @property
    def timesseq(self):
        '''
        Time series index
        Private property - can't be called directly
        '''
        return self.__timesseq

    @property
    def valuesseq(self):
        '''
        Time serires value
        Private property - can't be called directly
        '''
        return self.__valuesseq

    @property
    def times_to_index(self):
        '''
        map time index with integer index of the array
        Priviate property - can't be called directly
        '''
        return self.__times_to_index
    
    def times(self):
        '''
        Returns the times sequence.
        Parameters
        ----------
        None
        Returns
        -------
        Numpy array
            Time series times
        >>> t = [1, 1.5, 2, 2.5, 10]
        >>> v = [0, 2, -1, 0.5, 0]
        >>> a = TimeSeries(t, v)
        >>> a.times()
        array([  1. ,   1.5,   2. ,   2.5,  10. ])
        '''
        return self.timesseq

    def values(self):
        '''
        Returns the values sequence.
        Parameters
        ----------
        None
        Returns
        -------
        Numpy array
            Time series values
        >>> t = [1, 1.5, 2, 2.5, 10]
        >>> v = [0, 2, -1, 0.5, 0]
        >>> a = TimeSeries(t, v)
        >>> a.values()
        array([ 0. ,  2. , -1. ,  0.5,  0. ])
        '''
        return self.valuesseq

    def items(self):
        '''
        Returns sequence of (time, value) tuples.
        Parameters
        ----------
        None
        Returns
        -------
        Numpy array of tuples
            Time series time-value pairs
        >>> t = [1, 1.5, 2, 2.5, 10]
        >>> v = [0, 2, -1, 0.5, 0]
        >>> a = TimeSeries(t, v)
        >>> a.items()
        [(1.0, 0.0), (1.5, 2.0), (2.0, -1.0), (2.5, 0.5), (10.0, 0.0)]
        '''
        self.repOK(self.timesseq, self.valuesseq)
        return list(zip(self.timesseq, self.valuesseq))
    
    def __len__(self):
        '''
        >>> t = [1.5, 2, 2.5, 3, 10.5]
        >>> v = [1, 3, 0, 1.5, 1]
        >>> a = TimeSeries(t, v)
        >>> len(a)
        5
        >>> len(TimeSeries([], []))
        0
        '''
        self.repOK(self.timesseq, self.valuesseq)
        return len(self.timesseq)

    def __getitem__(self, time):
        '''
         >>> t = [1.5, 2, 2.5, 3, 10.5]
        >>> v = [1, 3, 0, 1.5, 1]
        >>> a = TimeSeries(t, v)
        >>> a[2.5]
        0
        '''
        self.repOK(self.timesseq, self.valuesseq)
        if time not in self.times_to_index:     
            raise IndexError('Time does not exist.')
        return self.valuesseq[self.times_to_index[float(time)]]

    def __setitem__(self, time, value):
        '''
        This does not allow us to extend the sequence.
        It only modifies an existing time's value.
        
        >>> t = [1.5, 2, 2.5, 3, 10.5]
        >>> v = [1, 3, 0, 1.5, 1]
        >>> a = TimeSeries(t, v)
        >>> a[1] = 12.0
        >>> a[1]
        12.0
        >>> a[5] = 9.0
        >>> a[5]
        9.0
        '''
        if time not in self.times_to_index:     
            raise IndexError('Time does not exist.')
        self.valuesseq[self.times_to_index[time]] = value
        self.repOK(self.timesseq, self.valuesseq)
        
    def __iter__(self):
        '''
        Iterates over values.
        
        >>> t = [1, 1.5, 2, 2.5, 10]
        >>> v = [0, 2, -1, 0.5, 0]
        >>> a = TimeSeries(t, v)
        >>> for val in a:
        ...     print(val)
        0.0
        2.0
        -1.0
        0.5
        0.0
        '''
        for v in self.valuesseq:
            yield v

    def itertimes(self):
        '''
        Iterates over the times array.
        Parameters
        ----------
        None
        Returns
        -------
        float(s)
            Iterator of time series times
        >>> t = [1, 1.5, 2, 2.5, 10]
        >>> v = [0, 2, -1, 0.5, 0]
        >>> a = TimeSeries(t, v)
        >>> for val in a.itertimes():
        ...     print(val)
        1.0
        1.5
        2.0
        2.5
        10.0
        '''
        for t in self.timesseq:
            yield t

    def itervalues(self):
        '''
        Iterates over the values array.
        Parameters
        ----------
        None
        Returns
        -------
        float(s)
            Iterator of time series values
        >>> t = [1, 1.5, 2, 2.5, 10]
        >>> v = [0, 2, -1, 0.5, 0]
        >>> a = TimeSeries(t, v)
        >>> for val in a.itervalues():
        ...     print(val)
        0.0
        2.0
        -1.0
        0.5
        0.0
        '''
        for v in self.valuesseq:
            yield v

    def iteritems(self):
        '''
        Iterates over the time-values pairs.
        Parameters
        ----------
        None
        Returns
        -------
        tuple(s) of floats
            Iterator of time series time-value pairs
        >>> t = [1, 1.5, 2, 2.5, 10]
        >>> v = [0, 2, -1, 0.5, 0]
        >>> a = TimeSeries(t, v)
        >>> for val in a.iteritems():
        ...     print(val)
        (1.0, 0.0)
        (1.5, 2.0)
        (2.0, -1.0)
        (2.5, 0.5)
        (10.0, 0.0)
        '''
        for t, v in zip(self.timesseq, self.valuesseq):
            yield t, v

    def __repr__(self):
        '''
        Only returns values without times.
        If more than 100 values, the output will be truncated with ellipses.
        '''
        class_name = type(self).__name__
        myrepr = reprlib.aRepr
        myrepr.maxlist = 100 # More than 100 then replace with ellipses
        components = myrepr.repr(self.valuesseq)
        components = components[components.find('['):]
        return '{}({})'.format(class_name, components)   
    
    def __str__(self):
        '''
        Only returns values without times.
        Shows the length of the time series, and first and last values.
        '''
        class_name = type(self).__name__
        first = 'N/A'
        last = 'N/A'
        if len(self) > 0:
            first = str(self.valuesseq[0])
            last = str(self.valuesseq[-1])
        return '%s\nLength: %d\nFirst (oldest): %s, Last (newest): %s' % (class_name, len(self), first, last)

IndentationError: unexpected indent (<ipython-input-1-5eb3ad39d73a>, line 191)

In [33]:
# Test this TimeSeries
t = [3., 1., 2.]
v = [4., 5., 6.]
z = TimeSeries(t,v)
z[3] = 8.0
z.items()
for zi in z.iteritems():
    print(zi)

(1.0, 5.0)
(2.0, 6.0)
(3.0, 8.0)


In [31]:
class ArrayTimeSeries(TimeSeries):
    '''
    Numpy array implementation of time series, extended from TimeSeries
    
    RepInv: Times and values must only include numbers, and must be same length.
    '''
    def __init__(self, times, values):
        '''
        Extends from TimeSeries to represent data internally with a numpy array.
        
        >>> t = [1.5, 2, 2.5, 3, 10.5]
        >>> v = [1, 3, 0, 1.5, 1]
        >>> z = TimeSeries(t,v)
        >>> z[3]
        1.5
        '''
        TimeSeries.__init__(self, times, values)
        # Represent data now as numpy array
        self.__timesseq = np.array(times, dtype=float)
        self.__valuesseq = np.array(values, dtype=float)
        self.__times_to_index = {t: i for i, t in enumerate(times)}

    # Override
    @property
    def timesseq(self):
        '''
        Time series index
        '''
        return self.__timesseq

    # Override
    @property
    def valuesseq(self):
        '''
        Time serires value
        '''
        return self.__valuesseq

    # Override
    @property
    def times_to_index(self):
        '''
        map time index with integer index of the array
        Priviate property - can't be called directly
        '''
        return self.__times_to_index

In [37]:
t = [1., 3., 2.]
v = [4., 5., 6.]
z = ArrayTimeSeries(t,v)
z[3] = 8.0
print(z)
for zi in z.iteritems():
    print(zi)

ArrayTimeSeries
Length: 3
First (oldest): 4.0, Last (newest): 6.0
(1.0, 4.0)
(3.0, 8.0)
(2.0, 6.0)
